## Helper functions for testing Kronos.

In [78]:
import requests
import pandas as pd
from datetime import datetime
import time

def download_binance_data(symbol='BTCUSDT', interval='1d', start_date='2020-01-01', end_date=None):
    """
    Download historical Bitcoin data from Binance API
    
    Args:
        symbol: Trading pair (default: BTCUSDT)
        interval: Time interval - 1m, 5m, 15m, 30m, 1h, 4h, 1d, 1w, 1M
        start_date: Start date in YYYY-MM-DD format
        end_date: End date in YYYY-MM-DD format (default: today)
    
    Returns:
        DataFrame with columns: timestamps, open, high, low, close, volume, amount
    """
    
    # Convert dates to milliseconds
    start_ts = int(datetime.strptime(start_date, '%Y-%m-%d').timestamp() * 1000)
    if end_date:
        end_ts = int(datetime.strptime(end_date, '%Y-%m-%d').timestamp() * 1000)
    else:
        end_ts = int(datetime.now().timestamp() * 1000)
    
    # Binance API endpoint
    url = 'https://api.binance.com/api/v3/klines'
    
    all_data = []
    current_ts = start_ts
    limit = 1000  # Max limit per request
    
    print(f"Downloading {symbol} data from {start_date} to {end_date or 'today'}...")
    
    while current_ts < end_ts:
        params = {
            'symbol': symbol,
            'interval': interval,
            'startTime': current_ts,
            'endTime': end_ts,
            'limit': limit
        }
        
        try:
            response = requests.get(url, params=params)
            response.raise_for_status()
            data = response.json()
            
            if not data:
                break
            
            all_data.extend(data)
            
            # Update current timestamp to the last candle's close time
            current_ts = data[-1][6] + 1
            
            print(f"Downloaded {len(all_data)} candles...", end='\r')
            
            # Respect rate limits
            time.sleep(0.1)
            
        except requests.exceptions.RequestException as e:
            print(f"\nError downloading data: {e}")
            break
    
    print(f"\nTotal candles downloaded: {len(all_data)}")
    
    # Convert to DataFrame
    df = pd.DataFrame(all_data, columns=[
        'open_time', 'open', 'high', 'low', 'close', 'volume',
        'close_time', 'quote_volume', 'trades', 'taker_buy_base',
        'taker_buy_quote', 'ignore'
    ])
    
    # Convert types
    df['open_time'] = pd.to_datetime(df['open_time'], unit='ms')
    for col in ['open', 'high', 'low', 'close', 'volume', 'quote_volume']:
        df[col] = df[col].astype(float)
    
    # Format according to requirements: timestamps, open, high, low, close, volume, amount
    result = pd.DataFrame({
        'timestamps': df['open_time'],
        'open': df['open'],
        'high': df['high'],
        'low': df['low'],
        'close': df['close'],
        'volume': df['volume'],
        'amount': df['quote_volume']  # Quote asset volume (USDT volume)
    })
    
    return result

# Options: 1m, 5m, 15m, 30m, 1h, 4h, 1d, 1w, 1M
interval = '1h'  # 4-hour intervals
df = download_binance_data(
    symbol='BTCUSDT',
    interval=interval,  
    start_date='2025-10-01',
    end_date=None  # None means today
)

# Save to CSV
output_file = f"bitcoin_historical_data.csv"
df.to_csv(output_file, index=False)
print(f"\nData saved to {output_file}")

Downloaded 429 candles...
Total candles downloaded: 429

Data saved to bitcoin_historical_data.csv
